In [ ]:
from diffusers import StableDiffusionXLInpaintPipeline
from PIL import Image, ImageChops
import torch
import time
import os
from IPython.display import display, clear_output
import cv2
import numpy as np
import random

# 设置设备
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# 加载模型
pipe = StableDiffusionXLInpaintPipeline.from_pretrained("stabilityai/sdxl-turbo", safety_checker=None)
# 如果上面这一行报错，即可能没法直接联网使用sdxl-turbo，那我们可以git sdxl-turbo然后通过本地使用，git clone https://huggingface.co/stabilityai/sdxl-turbo, 然后运行下面代码 
# pipe = StableDiffusionXLInpaintPipeline.from_pretrained("./sdxl-turbo", safety_checker=None)
pipe = pipe.to(device)




In [ ]:
ground_img = Image.open("ground.jpg")

# 定义图片大小调整函数
def resize(image, size):
    resized_image = image.resize((size, size))
    return resized_image

prompts = [
    "Wildfires, realistic style.",
    "The tallest towers of Shanghai, realistic style.",
    "Landfill, realistic style. ",
    "Hong Kong Light Pollution, realistic style.",
    "Plastic waste in oceans, realistic style.",
    "Industrial wastewater discharge, realistic style."
]


input_image = ground_img
mask_folder = "mask_design"  # 掩码文件夹路径

# 截取正方形区域的函数，根据白色部分中心位置确定裁剪区域
def crop_to_square(image, mask, size=2094):
    mask = mask.convert('L')
    binary_mask = mask.point(lambda p: p > 128 and 255)
    mask_array = np.array(binary_mask)
    white_indices = np.argwhere(mask_array == 255)
    y_min, x_min = white_indices.min(axis=0)
    y_max, x_max = white_indices.max(axis=0)
    center_y = (y_max + y_min) // 2
    start_y = max(center_y - size // 2, 0)
    if start_y + size > mask.height:
        start_y = mask.height - size
    cropped_image = image.crop((0, start_y, image.width, start_y + size))
    cropped_mask = mask.crop((0, start_y, image.width, start_y + size))
    return cropped_image, cropped_mask, start_y

# 定义图像到图像的转换函数
def img2img(src, mask, prompt):
    # 替换成实际的图像转换代码
    image = pipe(prompt=prompt, image=src, mask_image=mask, num_inference_steps=5, strength=1, guidance_scale=1).images[0]
    return image

# 合并图片，确保精确控制回贴位置
def merge_images(ground, img2img_result, mask, start_y):
    mask = mask.convert('1').resize(img2img_result.size)
    canvas = Image.new("RGB", ground.size)
    canvas.paste(ground, (0, 0))
    canvas.paste(img2img_result, (0, start_y), mask)
    return canvas

def display_image_cv(image):
    # 将PIL图像转换为OpenCV图像
    cv_image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    cv2.imshow('Image', cv_image)
    cv2.waitKey(10)  # 显示1毫秒以保持实时更新

def create_fade_frames(img1, img2, steps=10):
    img1_np = np.array(img1)
    img2_np = np.array(img2)
    fade_frames = []
    for i in range(steps):
        alpha = i / (steps - 1)
        beta = 1 - alpha
        blended = cv2.addWeighted(img1_np, beta, img2_np, alpha, 0)
        fade_frames.append(Image.fromarray(blended))
        display_image_cv(blended)
    return fade_frames

# 创建OpenCV窗口
cv2.namedWindow('Image', cv2.WINDOW_NORMAL)

In [ ]:
for index, _ in enumerate(os.listdir(mask_folder)):
    mask_path = f"{mask_folder}/{index}.png"
    mask_img = Image.open(mask_path)

    prompt = prompts[index % len(prompts)]

    # 裁剪图像到2094x2094，并调整大小到512x512
    square_ground, square_mask, start_y = crop_to_square(input_image, mask_img)
    resized_ground = resize(square_ground, 512)
    resized_mask = resize(square_mask, 512)

    # 图像到图像的转换
    img2img_result = img2img(resized_ground, resized_mask, prompt)

    # 将img2img的结果扩大到2094x2094
    img2img_large = img2img_result.resize((2094, 2094))

    # 合并回原始底图
    final_image = merge_images(input_image, img2img_large, square_mask, start_y)

    # 生成渐入渐出帧
    fade_frames = create_fade_frames(input_image, final_image, steps=20)


    # 更新输入图像为最新生成的图像
    input_image = final_image
    display_image_cv(input_image)

# 关闭OpenCV窗口
cv2.destroyAllWindows()